In [67]:
# import libraries
import numpy as np
import os
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
# from torchinfo import summary
# from contextlib import redirect_stdout

# from model.fcae import createLevel5Net as createNet

# from lib.evaluate import evaluate_total_predictions_set, display_mazes

%config InlineBackend.figure_formats = ['svg']
import matplotlib.pyplot as plt
%matplotlib inline

In [68]:
# select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [69]:
# load data
path_to_data = "../data/"

data_size = (10, 10)

# X_file = np.load(f"{path_to_data}100000x{data_size[0]}x{data_size[1]}_unsolved.npy")
# Y_file = np.load(f"{path_to_data}100000x{data_size[0]}x{data_size[1]}_solved.npy")
yHat_file = np.load(f"{path_to_data}predicted_20000x{data_size[0]}x{data_size[1]}.npy")
y_file = np.load(f"{path_to_data}label_20000x{data_size[0]}x{data_size[1]}.npy")

In [70]:
# # transform to tensor
# x_train = torch.tensor(x_train, dtype=torch.float32, device=device)
# y_train = torch.tensor(y_train, dtype=torch.float32, device=device)

yHat_test = torch.tensor(yHat_file, dtype=torch.float32, device=device)
y_test = torch.tensor(y_file, dtype=torch.float32, device=device)

In [71]:
print(yHat_test.shape)
print(y_test.shape)

torch.Size([20000, 1, 10, 10])
torch.Size([20000, 1, 10, 10])


In [78]:
def evaluate_found_path(answer: torch.Tensor, Y_test: torch.Tensor):
    start = find_gate(answer, 0.6, True)
    end = find_gate(answer, 0.6, False)

    print(start, end)

    if start is None or end is None:
        return False

    current = (start[0], start[1])
    visited = set()
    visited.add(current)

    max_moves = (
        answer.shape[0] * answer.shape[1]
    )  # Consider the grid size for max moves
    # max_moves = 17

    for i in range(max_moves):
        brightest_neighbour = find_brightest_neighbour(answer, current, visited, Y_test)
        if brightest_neighbour is None:
            return False
        current = (brightest_neighbour[0], brightest_neighbour[1])
        visited.add(current)
        if current == end:
            return True
    return False


def find_gate(answer: torch.Tensor, threshold: float, start: bool):
    rows, cols = answer.shape
    gate_candidates = (answer >= threshold).nonzero(as_tuple=True)  # Finde alle Positionen über dem Threshold

    if len(gate_candidates[0]) == 0:  # Kein Gate gefunden
        return None

    if start:
        return gate_candidates[0][0].item(), gate_candidates[1][0].item()  # Erster gefundener Kandidat
    else:
        return gate_candidates[0][-1].item(), gate_candidates[1][-1].item()  # Letzter gefundener Kandidat

def find_brightest_neighbour(
    answer: torch.Tensor,
    position: tuple[int, int],
    visited: set[tuple[int, int]],
    Y_test: torch.Tensor,
):
    rows, cols = answer.shape  # Get the actual number of rows and columns

    ind_up = (max(position[0] - 1, 0), position[1])
    ind_down = (min(position[0] + 1, rows - 1), position[1])
    ind_left = (position[0], max(position[1] - 1, 0))
    ind_right = (position[0], min(position[1] + 1, cols - 1))

    value_up = answer[ind_up[0]][ind_up[1]] if ind_up not in visited else -1
    value_down = answer[ind_down[0]][ind_down[1]] if ind_down not in visited else -1
    value_left = answer[ind_left[0]][ind_left[1]] if ind_left not in visited else -1
    value_right = answer[ind_right[0]][ind_right[1]] if ind_right not in visited else -1

    # Find the maximum value among the neighbours and return the corresponding index
    max_value = max(value_up, value_down, value_left, value_right)
    if max_value == -1:
        return None

    if max_value == value_up and Y_test[ind_up[0]][ind_up[1]] != 1.0:
        return ind_up
    elif max_value == value_down and Y_test[ind_down[0]][ind_down[1]] != 1.0:
        return ind_down
    elif max_value == value_left and Y_test[ind_left[0]][ind_left[1]] != 1.0:
        return ind_left
    elif max_value == value_right and Y_test[ind_right[0]][ind_right[1]] != 1.0:
        return ind_right

    return None


def evaluate_total_predictions_set(
    Y_hat: torch.Tensor, Y_test: torch.Tensor, verbose=True
):
    preds_: torch.Tensor = torch.zeros(Y_hat.shape[0], dtype=bool)

    # Dynamically determine the maze size from the data
    rows, cols = (
        Y_hat.shape[2],
        Y_hat.shape[3],
    )  # Assumes Y_hat has shape (batch_size, channels, rows, cols)

    wrong_predictions = []  # List to store images of wrong predictions

    for index in range(preds_.shape[0]):
        preds_[index] = evaluate_found_path(
            Y_hat[index].reshape(rows, cols), Y_test[index].numpy().reshape(rows, cols)
        )
        # Collect wrong predictions
        if not preds_[index]:
            wrong_predictions.append(Y_hat[index].reshape(rows, cols))

    total = preds_.shape[0]
    correct = preds_[preds_ == True].shape[0]
    correct_percentage = (correct / total) * 100

    if verbose:
        print("\n----------MODEL SUMMARY-----------")
        print("TOTAL SET SIZE: ", total)
        print("CORRECT GUESSES: ", correct)
        print("TOTALING TO ACCURACY%: ", correct_percentage)
        print("------------------------------------\n\n")

    if wrong_predictions:
        wrong_predictions_tensor = torch.stack(
            wrong_predictions
        )  # Stack wrong predictions into a tensor
    else:
        wrong_predictions_tensor = torch.empty(
            0
        )  # Return an empty tensor if no wrong predictions

    return correct_percentage, preds_, wrong_predictions_tensor

In [79]:
yHat_test = yHat_test[:1]
y_test = y_test[:1]

In [80]:
y_test

tensor([[[[3., 2., 2., 2., 2., 2., 2., 2., 2., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 2., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 2., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 2., 0.],
          [0., 0., 2., 2., 2., 2., 2., 2., 2., 0.],
          [0., 0., 2., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 2., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 2., 0., 3., 0., 0., 0., 0., 0.],
          [0., 0., 2., 2., 2., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]], device='cuda:0')

In [81]:
correct_percentage, _, _ = evaluate_total_predictions_set(yHat_test.cpu(), y_test.cpu())

(0, 0) (8, 6)

----------MODEL SUMMARY-----------
TOTAL SET SIZE:  1
CORRECT GUESSES:  1
TOTALING TO ACCURACY%:  100.0
------------------------------------




In [82]:
def find_gate_global(answer: torch.Tensor, threshold: float, start: bool):
    rows, cols = answer.shape
    gate_candidates = (answer >= threshold).nonzero(as_tuple=True)  # Finde alle Positionen über dem Threshold
    if start:
        for row, col in zip(*gate_candidates):
            if col == 0:  # Starte in der ersten Spalte
                return row, col
    else:
        for row, col in zip(*gate_candidates):
            if col == cols - 1:  # Suche in der letzten Spalte
                return row, col
    return None

start = find_gate_global(yHat_test[0, 0], threshold=2.5, start=True)
end = find_gate_global(yHat_test[0, 0], threshold=2.5, start=False)

print("Start Gate (Global Search):", start)
print("End Gate (Global Search):", end)


Start Gate (Global Search): (tensor(0, device='cuda:0'), tensor(0, device='cuda:0'))
End Gate (Global Search): None
